In [1]:
import pandas as pd
import numpy as np
import quickfix as qf
import quickfix42 as qf42

In [2]:
pd.options.display.notebook_repr_html = False

In [3]:
f = open('/Users/kapilsharma/GitHub/kaqt/python/notebooks/FIX.4.2-KAPILDTS-TTDEV13P.messages.current.log')
lines = f.readlines()
f.close()

In [4]:
lines = [x[24:][:-2].strip() for x in lines]

In [5]:
lines[:5]

['8=FIX.4.2\x019=89\x0135=A\x0134=1\x0149=KAPILDTS\x0152=20150508-17:11:23.099\x0156=TTDEV13P\x0196=12345678\x0198=0\x01108=30\x01141=Y\x0110=032\x01',
 '8=FIX.4.2\x019=00077\x0135=A\x0149=TTDEV13P\x0156=KAPILDTS\x0134=1\x0152=20150508-17:11:23.653\x01108=30\x01141=Y\x0198=0\x0110=088\x01',
 '8=FIX.4.2\x019=81\x0135=c\x0134=2\x0149=KAPILDTS\x0152=20150508-17:11:23.895\x0156=TTDEV13P\x01167=FUT\x01320=1\x0117000=Y\x0110=006\x01',
 '8=FIX.4.2\x019=00270\x0135=d\x0149=TTDEV13P\x0156=KAPILDTS\x0134=2\x0152=20150508-17:11:23.676\x0155=6B\x0148=00A0FP006BZ\x0110455=6BM5\x01167=FUT\x01207=CME\x0115=USD\x01320=1\x01322=1:0\x01200=201506\x0116451=0\x01393=29666\x01323=4\x0116452=1\x0116454=6.25\x0116552=1\x0116554=6.25\x0116456=0\x01146=0\x0118206=2\x0118203=CME\x0118203=CME-B\x01864=1\x01865=5\x01866=20150615\x0110=198\x01',
 '8=FIX.4.2\x019=110\x0135=3\x0134=3\x0149=KAPILDTS\x0152=20150508-17:11:23.913\x0156=TTDEV13P\x0145=2\x0158=Tag appears more than once\x01371=18203\x01372=d\x0110=189\x01

In [6]:
messages = [qf.Message(x) for x in lines]

In [7]:
messages = pd.Series(messages)

In [8]:
sec_defs = messages[messages.apply(lambda x : x.getHeader().getField(35) == 'd')] 

In [9]:
sec_defs.tail()

33344    8=FIX.4.29=27935=d34=2966349=TTDEV13P52=2...
33345    8=FIX.4.29=26935=d34=2966449=TTDEV13P52=2...
33346    8=FIX.4.29=26935=d34=2966549=TTDEV13P52=2...
33347    8=FIX.4.29=27135=d34=2966649=TTDEV13P52=2...
33348    8=FIX.4.29=27135=d34=2966749=TTDEV13P52=2...
dtype: object

In [10]:
# I  need SecurityID, Symbol, SecurityDesc, SecurityType, MaturityMonthYear , SecurityExchange , TotNoRelatedSym , NoEvents ,EventType , EventDate , all TT tags 

In [11]:
def get_clean_field(x, tag):
    ret_val = np.nan
    if x.isSetField(tag) is True:
        ret_val = x.getField(tag)
    return ret_val

def getdata_for_field(msgs, dict, key):
    return msgs.apply(lambda x : get_clean_field(x, dict[key]))

In [12]:
tags = {
    'SecurityID' : 48,
    'Symbol' : 55,
    'SecurityDesc' : 107,
    'SecurityType' : 167,
    'MaturityMonthYear' : 200,
    'SecurityExchange' : 207,
    'EventType' : 865,
    'EventDate' : 866,
    'ContractSymbol' : 10455,
    'PriceDisplayType' : 16451,
    'TickSize' : 16452,
    'PointValue' : 16454,
    'ExchTickSize' : 16552,
    'ExchPointValue' : 16554,
}


In [25]:
data = {}
for key in tags:
    data[key] = getdata_for_field(sec_defs, tags, key)

In [14]:
data = pd.DataFrame(data)

In [67]:
def get_df_row(msg):
    security_id = get_clean_field(msg, tags['SecurityID'])
    symbol = get_clean_field(msg, tags['Symbol'])
    security_desc = get_clean_field(msg, tags['SecurityDesc'])
    security_type = get_clean_field(msg, tags['SecurityType'])
    maturity_month_year = get_clean_field(msg, tags['MaturityMonthYear'])
    security_exchange = get_clean_field(msg, tags['SecurityExchange'])
    event_type = get_clean_field(msg, tags['EventType'])
    event_date = get_clean_field(msg, tags['EventDate'])
    contract_symbol = get_clean_field(msg, tags['ContractSymbol'])
    price_display_format = get_clean_field(msg, tags['PriceDisplayType'])
    tick_size = get_clean_field(msg, tags['TickSize'])
    point_value = get_clean_field(msg, tags['PointValue'])
    exch_tick_size = get_clean_field(msg, tags['ExchTickSize'])
    exch_point_value = get_clean_field(msg, tags['ExchPointValue'])
    row_data = {
        'SecurityID' : security_id,
        'Symbol' : symbol,
        'SecurityDesc' : security_desc,
        'SecurityType' : security_type,
        'MaturityMonthYear' : maturity_month_year,
        'SecurityExchange' : security_exchange,
        'EventType' : event_type,
        'EventDate' : event_date,
        'ContractSymbol' : contract_symbol,
        'PriceDisplayType' : price_display_format,
        'TickSize' : tick_size,
        'PointValue' : point_value,
        'ExchTickSize' : exch_tick_size,
        'ExchPointValue' : exch_point_value
    }
    return row_data

In [68]:
security_definitions = pd.DataFrame(columns=tags.keys())
security_definitions

Empty DataFrame
Columns: [PriceDisplayType, MaturityMonthYear, SecurityDesc, ExchTickSize, TickSize, ExchPointValue, ContractSymbol, EventType, Symbol, SecurityExchange, SecurityType, SecurityID, PointValue, EventDate]
Index: []

In [72]:
all_data = []
for sd in sec_defs:
    row_data = get_df_row(sd)
    all_data.append(row_data)

In [66]:
len(security_definitions)

0